In [105]:
import pandas as pd
from pprint import pprint
from collections import Counter as C
import os
import matplotlib.pyplot as plt

In [106]:
out0 = pd.read_csv('output_0.csv')
out1 = pd.read_csv('output_1.csv')
out2 = pd.read_csv('output_2.csv')
out3 = pd.read_csv('output_3.csv')
out4 = pd.read_csv('output_4.csv')
out5 = pd.read_csv('output_5.csv')
out6 = pd.read_csv('output_6.csv')
# out7 = pd.read_csv('output_7.csv')

In [107]:
out = pd.DataFrame(columns = ['ImageID']+[f'ans{i}' for i in range(7)])
mask = pd.DataFrame(columns = ['ImageID']+[f'mask{i}' for i in range(7)])
gen = pd.DataFrame(columns = ['ImageID']+[f'gen{i}' for i in range(7)])
age = pd.DataFrame(columns = ['ImageID']+[f'age{i}' for i in range(7)])
decode_ans = pd.DataFrame(columns = ['ImageID', 'mask','gen','age'])

In [108]:
out['ImageID'] = out0['ImageID']
mask['ImageID'] = out0['ImageID']
gen['ImageID'] = out0['ImageID']
age['ImageID'] = out0['ImageID']
decode_ans['ImageID'] = out0['ImageID']
out['ans0'] = out0['ans']
out['ans1'] = out1['ans']
out['ans2'] = out2['ans']
out['ans3'] = out3['ans']
out['ans4'] = out4['ans']
out['ans5'] = out5['ans']
out['ans6'] = out6['ans']
# out['ans7'] = out7['ans']

In [96]:
def encode_multi_class(mask_label, gender_label, age_label):
    return mask_label * 6 + gender_label * 3 + age_label
def decode_multi_class(multi_class_label):
    mask_label = (multi_class_label // 6) % 3
    gender_label = (multi_class_label // 3) % 2
    age_label = multi_class_label % 3
    return [mask_label, gender_label, age_label]

In [110]:
mask_decodes = []
gen_decodes = []
age_decodes = []
for i in range(len(out0)):
    row = out.loc[i, :]
    mask_decode = []
    gen_decode = []
    age_decode = []
    for j in range(7):
        d = decode_multi_class(row[f'ans{j}'])
        mask_decode.append(d[0])
        gen_decode.append(d[1])
        age_decode.append(d[2])
    mask_decodes.append(mask_decode)
    gen_decodes.append(gen_decode)
    age_decodes.append(age_decode)

In [113]:
mask[[f'mask{i}' for i in range(7)]] = mask_decodes
gen[[f'gen{i}' for i in range(7)]] = gen_decodes
age[[f'age{i}' for i in range(7)]] = age_decodes

In [119]:
mask_ans = []
mask_check = []
for i in range(len(mask)):
    row = mask.loc[i,:][[f'mask{i}' for i in range(7)]]
    cnt = C(row).most_common(2)
    first = cnt[0]
    mask_ans.append(first[0])
    if len(cnt) >= 2:
        second = cnt[1]
        if first[1] == second[1]:
            mask_check.append((i, first[0], second[0]))

In [120]:
gen_ans = []
gen_check = []
for i in range(len(gen)):
    row = gen.loc[i,:][[f'gen{i}' for i in range(7)]]
    cnt = C(row).most_common(2)
    first = cnt[0]
    gen_ans.append(first[0])
    if len(cnt) >= 2:
        second = cnt[1]
        if first[1] == second[1]:
            gen_check.append((i, first[0], second[0]))

In [121]:
age_ans = []
age_check = []
for i in range(len(age)):
    row = age.loc[i,:][[f'age{i}' for i in range(7)]]
    cnt = C(row).most_common(2)
    first = cnt[0]
    age_ans.append(first[0])
    if len(cnt) >= 2:
        second = cnt[1]
        if first[1] == second[1]:
            age_check.append((i, first[0], second[0]))

In [122]:
for i in [j[0] for j in age_check]:
    age_ans[i] = 1

In [123]:
decode_ans['mask'] = mask_ans
decode_ans['gen'] = gen_ans
decode_ans['age'] = age_ans

In [125]:
decode_ans['ans'] = decode_ans['mask']*6+decode_ans['gen']*3+decode_ans['age']

In [127]:
answer = decode_ans[['ImageID', 'ans']]

In [103]:
answer.to_csv('answer.csv', index = False)